Modified from http://www.johnmyleswhite.com/notebook/2013/01/07/symbolic-differentiation-in-julia/

In [135]:
expr = :(x + 1)

:(x + 1)

In [136]:
typeof(expr)

Expr

In [137]:
expr |> dump

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Symbol x
    3: Int64 1
  typ: Any


In [138]:
@doc Expr

No documentation found.

**Summary:**

```julia
type Expr <: Any
```

**Fields:**

```julia
head :: Symbol
args :: Array{Any,1}
typ  :: Any
```


In [139]:
expr2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [140]:
expr2 |> eval

2

In [141]:
# Constant
differentiate(x::Number, target::Symbol) = 0

differentiate (generic function with 3 methods)

In [142]:
differentiate(5, :x)

0

In [143]:
# Just one symbol
function differentiate(s::Symbol, target::Symbol)
    if s == target
        return 1
    else
        return 0
    end
end

differentiate (generic function with 3 methods)

In [144]:
differentiate(:x, :x)

1

In [145]:
differentiate(:y, :x)

0

In [146]:
# Sum
function differentiate_sum(ex::Expr, target::Symbol)
    #@printf "diff[+]: %s\n" ex
    n = length(ex.args)
    new_args = Array(Any, n)
    new_args[1] = :+
    for i in 2:n
        new_args[i] = differentiate(ex.args[i], target)
    end
    return Expr(:call, new_args...)
end

differentiate_sum (generic function with 1 method)

In [147]:
differentiate_sum(:(x + y), :x)

:(1 + 0)

In [148]:
function differentiate_subtraction(ex::Expr, target::Symbol)
    #@printf "diff[-]: %s\n" ex
    n = length(ex.args)
    new_args = Array(Any, n)
    new_args[1] = :-
    for i in 2:n
        new_args[i] = differentiate(ex.args[i], target)
    end
    return Expr(:call, new_args...)
end

differentiate_subtraction (generic function with 1 method)

In [149]:
differentiate_subtraction(:(x - y), :x)

:(1 - 0)

In [150]:
function differentiate_product(ex::Expr, target::Symbol)
    #@printf "diff[*]: %s\n" ex
    n = length(ex.args)
    res_args = Array(Any, n)
    res_args[1] = :+
    for i in 2:n
       new_args = Array(Any, n)
       new_args[1] = :*
       for j in 2:n
           if j == i
               new_args[j] = differentiate(ex.args[j], target)
           else
               new_args[j] = ex.args[j]
           end
       end
       res_args[i] = Expr(:call, new_args...)
    end
    return Expr(:call, res_args...)
end

differentiate_product (generic function with 1 method)

In [151]:
differentiate_product(:(x * x), :x)

:(1x + x * 1)

In [152]:
function differentiate_quotient(ex::Expr, target::Symbol)
    #@printf "diff[/]: %s\n" ex
    return Expr(:call,
                :/,
                Expr(:call,
                    :-,
                    Expr(:call,
                        :*,
                        differentiate(ex.args[2], target),
                        ex.args[3]
                    ),
                    Expr(:call,
                        :*,
                        ex.args[2],
                        differentiate(ex.args[3], target)
                    )
                ),
                Expr(:call,
                    :^,
                    ex.args[3],
                    2
                )
            )
end

differentiate_quotient (generic function with 1 method)

In [153]:
differentiate_quotient(:(x / y), :y)

:((0y - x * 1) / y ^ 2)

In [154]:
differentiate_lookup = Dict(
  :+ => differentiate_sum,
  :- => differentiate_subtraction,
  :* => differentiate_product,
  :/ => differentiate_quotient
)

Dict{Symbol,Function} with 4 entries:
  :/ => differentiate_quotient
  :+ => differentiate_sum
  :* => differentiate_product
  :- => differentiate_subtraction

In [155]:
function differentiate(ex::Expr, target::Symbol)
    # @printf "diff: %s\n" ex
    if ex.head == :call
        if haskey(differentiate_lookup, ex.args[1])
            return differentiate_lookup[ex.args[1]](ex, target)
        else
            error("Don't know how to differentiate $(ex.args[1])")
        end
    else
        return differentiate(ex.head)
    end
end

differentiate (generic function with 3 methods)

In [156]:
differentiate(:(1 + x + x * x), :x)

:(0 + 1 + (1x + x * 1))

In [160]:
differentiate(:(1 + z + x * z + x * z * z), :z)

:(0 + 1 + (0z + x * 1) + (0 * z * z + x * 1 * z + x * z * 1))